In [267]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [268]:
# Get test_q_keys
import random
import pandas as pd
from src.utils import get_file_from_s3

survey_name = "American_Trends_Panel_W34"
random.seed(42)
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
q_keys = list(survey_df['key'])
test_q_keys = random.choices(q_keys, k=5)

print(test_q_keys)

['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']


In [269]:


import random
random.seed(42)

test_q_keys_mapping = {
    "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
    "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
    "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
    "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
    "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
}


# Hyper parameters

In [270]:
# survey_name = "American_Trends_Panel_W26"

# def skip_exp(filename):
#     if 'sonnetpred' not in filename and 'haikupred' not in filename: return True
#     if all(_ not in filename for _ in ['v11', 'v10']): return True
#     if any(_ in filename for _ in ['hcbdeu', 'hcaic']): return True
#     if ('v11' in filename) and ('bds' not in filename): return True
#     if ('v11' in filename) and ('persona' not in filename): return True
#     print('including: ', filename)
#     return False

# def get_exp(filename):
#     if filename[-5:] == '.json': filename = filename[:-5]
#     exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'v10', 'v11', 'v12', 'haikupred', 'sonnetpred']]
#     final = '_'.join(exp_elements)
#     # print(filename, '->', final)
#     return final


In [271]:

survey_name = "American_Trends_Panel_W26"
survey_name = "American_Trends_Panel_W27"
survey_name = "American_Trends_Panel_W29"
# survey_name = "American_Trends_Panel_W32"
# survey_name = "American_Trends_Panel_W34"

def skip_exp(filename):
    if ('0828' not in filename or 'sonnetpred' not in filename): return True
    # if 'prompt1' not in filename: return True
    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'prompt1']]
    final = '_'.join(exp_elements)
    # print(filename, '->', final)
    return final

# Get Accuracy

In [272]:
import json
import os
import pandas as pd
import numpy as np

res = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
    if skip_exp(filename): continue
    exp = get_exp(filename)
    # exp = filename
    if exp not in res:
        res[exp] = []
        
    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)
    # res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", f"{sum([_['is_correct'] for _ in data])/len(data):.4f}"]
    res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]


print(res)
df = pd.DataFrame(res)
df = df[sorted(df.columns)]
df = df.transpose()

# df['acc_avg'] = df.apply(lambda x: np.mean(x[[1,3,5]]), axis=1)
# df['acc_var'] = df.apply(lambda x: np.var(x[[1,3,5]]), axis=1)
# df['acc_std'] = df.apply(lambda x: np.std(x[[1,3,5]]), axis=1)
# df['acc_avg%'] = df.apply(lambda x: f"{100*x['acc_avg']:.2f}%", axis=1)
# df['acc_std%'] = df.apply(lambda x: f"{100*np.std(x[[1,3,5]]):.2f}%", axis=1)

acc_df = df
acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
acc_df.sort_values(by='acc', ascending=False)

{'persona': ['=689/1600', 0.430625], 'demo': ['=759/1600', 0.474375], 'demo_persona': ['=705/1600', 0.440625], 'history_persona': ['=792/1600', 0.495], 'history_demo_persona': ['=808/1600', 0.505], 'history': ['=952/1600', 0.595], 'history_demo': ['=908/1600', 0.5675]}


,acc_cnt,acc
history,=952/1600,0.595
history_demo,=908/1600,0.5675
history_demo_persona,=808/1600,0.505
history_persona,=792/1600,0.495
demo,=759/1600,0.474375
demo_persona,=705/1600,0.440625
persona,=689/1600,0.430625


# Get 1-WD

In [273]:
import pandas as pd
from src.utils import get_file_from_s3

test_q_keys = test_q_keys_mapping[survey_name]
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

wd_question_mappings = {row['key']: dict(zip(eval(row['references']), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
wd_question_mappings

{'NOWSMK_NHIS_W29': {'Every day': 1.0, 'Some days': 2.0, 'Not at all': 3.0},
 'HELPHURTA_W29': {'Has done more to help my job or career': 1.0,
  'Has done more to hurt my job or career': 2.0,
  "Hasn't made much of a difference": 1.5},
 'HELPHURTE_W29': {'Has done more to help my job or career': 1.0,
  'Has done more to hurt my job or career': 2.0,
  "Hasn't made much of a difference": 1.5},
 'TRAITSD_W29': {'Very well': 1.0,
  'Somewhat well': 2.0,
  'Not too well': 3.0,
  'Not at all well': 4.0},
 'GIRLSF2A_W29': {'Too much emphasis': 1.0,
  'Too little emphasis': 2.0,
  'About right': 1.5}}

## Question level

In [274]:
import random
from collections import Counter

records = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
    if skip_exp(filename): continue

    print(filename)
    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)
    
    exp = get_exp(filename)

    res = []
    pred_missing_cnt = 0
    for entry in data:
        q_idx = entry['q_idx']
        q_key = test_q_keys[q_idx]
        pos_mapping = wd_question_mappings[q_key]
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        if gold_answer not in pos_mapping: continue
        if prediction not in pos_mapping:
            pred_missing_cnt += 1
            if prediction not in entry['references']:
                print(f'Prediction: |{prediction}|, Gold: |{gold_answer}|')
            continue
            prediction = random.choice(list(pos_mapping.keys()))
        pred_pos = pos_mapping[prediction]
        gold_pos = pos_mapping[gold_answer]
        res.append(abs(pred_pos-gold_pos))
    records[exp] = [Counter(res), np.mean(res), pred_missing_cnt]

q_wd_df = pd.DataFrame(records)
q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc"})
q_wd_df

date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_persona_run1.json
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_demo_run1.json
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_demo_persona_run1.json
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_history_persona_run1.json
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_history_demo_persona_run1.json
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_history_run1.json
Prediction: |Describes me well|, Gold: |Not too well|
Prediction: |Describes me well|, Gold: |Very well|
Prediction: |Describes me well|, Gold: |Somewhat well|
date0828_v10post_midterm_American_Trends_Panel_W29_sonnetpred_history_demo_run1.json


,q_1wd_cnt,q_1wd,q_1wd_pmc
demo,"{0.0: 759, 1.0: 332, 0.5: 424, 3.0: 3, 2.0: 67}",0.433438,0
demo_persona,"{0.0: 705, 1.0: 352, 0.5: 438, 3.0: 6, 2.0: 84}",0.477603,0
history,"{0.0: 950, 1.0: 285, 0.5: 289, 3.0: 2, 2.0: 56}",0.346081,3
history_demo,"{0.0: 908, 1.0: 291, 0.5: 326, 3.0: 2, 2.0: 58}",0.363407,0
history_demo_persona,"{0.0: 808, 1.0: 335, 0.5: 369, 2.0: 73}",0.419874,0
history_persona,"{0.0: 792, 1.0: 337, 0.5: 387, 2.0: 69}",0.421767,0
persona,"{0.0: 689, 0.5: 464, 1.0: 349, 3.0: 3, 2.0: 80}",0.473186,0


## Survey level

In [275]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wasserstein_distance


def get_1wd_from_df(df, ref_ord_mapping):
    pred_df = df[df['Type'] == 'Prediction']
    pred_dict = {row['Answer']: row['Count'] for idx, row in pred_df.iterrows() if row['Answer'] != 'Refused'}
    gold_df = df[df['Type'] == 'Gold']
    gold_dict = {row['Answer']: row['Count'] for idx, row in gold_df.iterrows() if row['Answer'] != 'Refused'}

    pred_dict = {k: v for k, v in pred_dict.items() if k in ref_ord_mapping}

    distribution_1 = pred_dict
    distribution_2 = gold_dict

    # Get the union of keys
    all_keys = sorted(list(set(distribution_1.keys()).union(set(distribution_2.keys()))))

    # Create aligned frequency arrays
    freqs_1 = np.array([distribution_1.get(key, 0) for key in all_keys])
    freqs_2 = np.array([distribution_2.get(key, 0) for key in all_keys])

    # Normalize the frequencies to get probabilities
    probs_1 = freqs_1 / freqs_1.sum()
    probs_2 = freqs_2 / freqs_2.sum()

    # # Compute the cumulative distribution functions
    cdf_1 = np.cumsum(probs_1)
    cdf_2 = np.cumsum(probs_2)

    # Compute the 1-Wasserstein distance

    positions = [ref_ord_mapping[k] for k in all_keys]  # Positions in the sorted key space

    wd_distance = wasserstein_distance(positions, positions, u_weights=probs_1, v_weights=probs_2)
    # wd_distance = wasserstein_distance(positions, positions, u_weights=cdf_1, v_weights=cdf_2)
    # print(positions)
    # print(wd_distance)

    return wd_distance


# Load the data
record = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):

    if skip_exp(filename): continue

    exp = get_exp(filename)
    res = [{"pred": [], "gold": []} for _ in range(5)]
    if exp not in record: record[exp] = []

    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)

    for entry in data:
        q_idx = entry['q_idx']
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        res[q_idx]['pred'].append(prediction)
        res[q_idx]['gold'].append(gold_answer)

    # Create dataframes for visualization
    dataframes = []
    for i in range(5):
        df_pred = pd.Series(res[i]['pred']).value_counts().reset_index()
        df_pred.columns = ['Answer', 'Count']
        df_pred['Type'] = 'Prediction'
        df_pred['Question'] = f'Question {i+1}'
        
        df_gold = pd.Series(res[i]['gold']).value_counts().reset_index()
        df_gold.columns = ['Answer', 'Count']
        df_gold['Type'] = 'Gold'
        df_gold['Question'] = f'Question {i+1}'
        
        dataframes.append(pd.concat([df_pred, df_gold]))

    wd = []
    for q_idx, df in enumerate(dataframes):
        key = test_q_keys[q_idx]
        ref_ord_mapping = wd_question_mappings[key]
        wd.append(get_1wd_from_df(df, ref_ord_mapping))

    record[exp].append(np.mean(wd) * 100)

s_wd_df = pd.DataFrame(record)
s_wd_df = s_wd_df[sorted(s_wd_df.columns)].transpose()

# s_wd_df['wd_avg'] = s_wd_df.apply(lambda x: f"{np.mean(x[[0,1,2]]):.2f}", axis=1)
# s_wd_df['wd_std'] = s_wd_df.apply(lambda x: f"{np.std(x[[0,1,2]]):.2f}", axis=1)
s_wd_df = s_wd_df.rename(columns={0: 's_1wd'})
s_wd_df

,s_1wd
demo,14.981421
demo_persona,21.653348
history,19.142894
history_demo,17.472397
history_demo_persona,25.216336
history_persona,26.911592
persona,24.284502


# Full Results

In [276]:
merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
# merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
# merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
merged_df = merged_df.sort_values(by='acc', ascending=False)

# merged_df.sort_values(by='acc_avg', ascending=False)
# merged_df.sort_values(by='wd_avg', ascending=True)
# merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

merged_df

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc
history,=952/1600,0.595,"{0.0: 950, 1.0: 285, 0.5: 289, 3.0: 2, 2.0: 56}",0.346081,3
history_demo,=908/1600,0.5675,"{0.0: 908, 1.0: 291, 0.5: 326, 3.0: 2, 2.0: 58}",0.363407,0
history_demo_persona,=808/1600,0.505,"{0.0: 808, 1.0: 335, 0.5: 369, 2.0: 73}",0.419874,0
history_persona,=792/1600,0.495,"{0.0: 792, 1.0: 337, 0.5: 387, 2.0: 69}",0.421767,0
demo,=759/1600,0.474375,"{0.0: 759, 1.0: 332, 0.5: 424, 3.0: 3, 2.0: 67}",0.433438,0
demo_persona,=705/1600,0.440625,"{0.0: 705, 1.0: 352, 0.5: 438, 3.0: 6, 2.0: 84}",0.477603,0
persona,=689/1600,0.430625,"{0.0: 689, 0.5: 464, 1.0: 349, 3.0: 3, 2.0: 80}",0.473186,0


**For printing:**

In [277]:
merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
merged_df

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc
history,'=952/1600,0.595,"{0.0: 950, 1.0: 285, 0.5: 289, 3.0: 2, 2.0: 56}",0.346081,3
history_demo,'=908/1600,0.5675,"{0.0: 908, 1.0: 291, 0.5: 326, 3.0: 2, 2.0: 58}",0.363407,0
history_demo_persona,'=808/1600,0.505,"{0.0: 808, 1.0: 335, 0.5: 369, 2.0: 73}",0.419874,0
history_persona,'=792/1600,0.495,"{0.0: 792, 1.0: 337, 0.5: 387, 2.0: 69}",0.421767,0
demo,'=759/1600,0.474375,"{0.0: 759, 1.0: 332, 0.5: 424, 3.0: 3, 2.0: 67}",0.433438,0
demo_persona,'=705/1600,0.440625,"{0.0: 705, 1.0: 352, 0.5: 438, 3.0: 6, 2.0: 84}",0.477603,0
persona,'=689/1600,0.430625,"{0.0: 689, 0.5: 464, 1.0: 349, 3.0: 3, 2.0: 80}",0.473186,0


# Get distribution of responses by question

In [278]:
viz_filename = f"date0826_v10post_midterm_{survey_name}_history_run1.json"

In [279]:
import pandas as pd
from src.utils import get_file_from_s3
from tabulate import tabulate

test_q_keys = test_q_keys_mapping[survey_name]
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
sub_df = survey_df[survey_df['key'].isin(test_q_keys)][['key', 'question', 'references']]

print(tabulate(sub_df, headers="keys"))

    key              question                                                                                                                                                                                                    references
--  ---------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------
 1  NOWSMK_NHIS_W29  Do you now smoke cigarettes                                                                                                                                                                                 ['Every day', 'Some days', 'Not at all', 'Refused']
17  HELPHURTA_W29    Thinking about your work life, has the growing number of immigrants working in this country done more to help

In [280]:
q_key_to_q_text = dict(zip(sub_df['key'], sub_df['question']))
q_texts = [q_key_to_q_text[key] for key in test_q_keys]

print(q_key_to_q_text)
print(q_texts)

{'NOWSMK_NHIS_W29': 'Do you now smoke cigarettes', 'HELPHURTA_W29': "Thinking about your work life, has the growing number of immigrants working in this country done more to help or hurt your job or career, or hasn't it made much of a difference for your job or career?", 'HELPHURTE_W29': "Thinking about your work life, has automation of jobs through new technology in the workplace done more to help or hurt your job or career, or hasn't it made much of a difference for your job or career?", 'TRAITSD_W29': 'How well, if at all, do the following words or phrases describe you? Assertive', 'GIRLSF2A_W29': 'When it comes to raising girls, would you say there is too much emphasis or too little emphasis on encouraging girls to be leaders these days, or is it about right?'}
['How well, if at all, do the following words or phrases describe you? Assertive', 'Do you now smoke cigarettes', "Thinking about your work life, has automation of jobs through new technology in the workplace done more to he

In [281]:
import textwrap
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
res = [{"pred": [], "gold": []} for _ in range(5)]
for filename in os.listdir(f'opinions_qa/output/{survey_name}'):

    with open(f"opinions_qa/output/{survey_name}/{viz_filename}", 'r') as f:
        data = json.load(f)

    for entry in data:
        q_idx = entry['q_idx']
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        res[q_idx]['pred'].append(prediction)
        res[q_idx]['gold'].append(gold_answer)



# Create dataframes for visualization
dataframes = []
for i in range(5):
    df_pred = pd.Series(res[i]['pred']).value_counts().reset_index()
    df_pred.columns = ['Answer', 'Count']
    df_pred['Type'] = 'Prediction'
    df_pred['Question'] = f'Question {i+1}'
    
    df_gold = pd.Series(res[i]['gold']).value_counts().reset_index()
    df_gold.columns = ['Answer', 'Count']
    df_gold['Type'] = 'Gold'
    df_gold['Question'] = f'Question {i+1}'
    
    dataframes.append(pd.concat([df_pred, df_gold]))

# Plotting
sns.set(style="whitegrid")

# Function to plot the data
def plot_question(df, question_idx):
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=df, x="Answer", y="Count", hue="Type", palette=["#a6cee3", "#1f78b4"], alpha=0.7)
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.annotate(f'{int(height)}', 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points',
                        fontsize=10)
    wrapped_title = "\n".join(textwrap.wrap(q_texts[question_idx], width=80))
    plt.title(wrapped_title, fontsize=16, pad=20)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel('Answer', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Type', title_fontsize='13', fontsize='11')
    plt.show()

# Draw the graphs one by one
for i in range(5):
    plot_question(dataframes[i], i)


FileNotFoundError: [Errno 2] No such file or directory: 'opinions_qa/output/American_Trends_Panel_W29/date0826_v10post_midterm_American_Trends_Panel_W29_history_run1.json'